In [11]:
#Importing Libraries
import pandas as pd
import numpy as np

pd.set_option("display.max_columns", None)
print("Libraries loaded")


Libraries loaded


In [7]:
# Load Dataset
df = pd.read_csv("insurance_claims.csv")
print(df.shape)
df.head()

(1000, 40)


,months_as_customer,age,policy_number,policy_bind_date,policy_state,policy_csl,policy_deductable,policy_annual_premium,umbrella_limit,insured_zip,insured_sex,insured_education_level,insured_occupation,insured_hobbies,insured_relationship,capital-gains,capital-loss,incident_date,incident_type,collision_type,incident_severity,authorities_contacted,incident_state,incident_city,incident_location,incident_hour_of_the_day,number_of_vehicles_involved,property_damage,bodily_injuries,witnesses,police_report_available,total_claim_amount,injury_claim,property_claim,vehicle_claim,auto_make,auto_model,auto_year,fraud_reported,_c39
0,328,48,521585,2014-10-17,OH,250/500,1000,1406.91,0,466132,MALE,MD,craft-repair,sleeping,husband,53300,0,2015-01-25,Single Vehicle Collision,Side Collision,Major Damage,Police,SC,Columbus,9935 4th Drive,5,1,YES,1,2,YES,71610,6510,13020,52080,Saab,92x,2004,Y,NaN
1,228,42,342868,2006-06-27,IN,250/500,2000,1197.22,5000000,468176,MALE,MD,machine-op-inspct,reading,other-relative,0,0,2015-01-21,Vehicle Theft,?,Minor Damage,Police,VA,Riverwood,6608 MLK Hwy,8,1,?,0,0,?,5070,780,780,3510,Mercedes,E400,2007,Y,NaN
2,134,29,687698,2000-09-06,OH,100/300,2000,1413.14,5000000,430632,FEMALE,PhD,sales,board-games,own-child,35100,0,2015-02-22,Multi-vehicle Collision,Rear Collision,Minor Damage,Police,NY,Columbus,7121 Francis Lane,7,3,NO,2,3,NO,34650,7700,3850,23100,Dodge,RAM,2007,N,NaN
3,256,41,227811,1990-05-25,IL,250/500,2000,1415.74,6000000,608117,FEMALE,PhD,armed-forces,board-games,unmarried,48900,-62400,2015-01-10,Single Vehicle Collision,Front Collision,Major Damage,Police,OH,Arlington,6956 Maple Drive,5,1,?,1,2,NO,63400,6340,6340,50720,Chevrolet,Tahoe,2014,Y,NaN
4,228,44,367455,2014-06-06,IL,500/1000,1000,1583.91,6000000,610706,MALE,Associate,sales,board-games,unmarried,66000,-46000,2015-02-17,Vehicle Theft,?,Minor Damage,NaN,NY,Arlington,3041 3rd Ave,20,1,NO,0,1,NO,6500,1300,650,4550,Accura,RSX,2009,N,NaN


In [13]:
#Audit Dataset
print("Nulls by column:")
print(df.isna().sum())

print("\nDtypes:")
print(df.dtypes)

print("\nDuplicates:", df.duplicated().sum())

Nulls by column:
months_as_customer                0
age                               0
policy_number                     0
policy_bind_date                  0
policy_state                      0
policy_csl                        0
policy_deductable                 0
policy_annual_premium             0
umbrella_limit                    0
insured_zip                       0
insured_sex                       0
insured_education_level           0
insured_occupation                0
insured_hobbies                   0
insured_relationship              0
capital-gains                     0
capital-loss                      0
incident_date                     0
incident_type                     0
collision_type                    0
incident_severity                 0
authorities_contacted            91
incident_state                    0
incident_city                     0
incident_location                 0
incident_hour_of_the_day          0
number_of_vehicles_involved       0
property_da

In [15]:
#Data Cleaning
df_clean = df.copy()

# Drop empty column
if "_c39" in df_clean.columns and df_clean["_c39"].isna().sum() == len(df_clean):
    df_clean = df_clean.drop(columns=["_c39"])

# Replace "?" with NaN
df_clean = df_clean.replace("?", pd.NA)

# Convert dates
for col in ["policy_bind_date", "incident_date"]:
    if col in df_clean.columns:
        df_clean[col] = pd.to_datetime(df_clean[col], errors="coerce")

# Strip spaces
for col in df_clean.select_dtypes(include="object").columns:
    df_clean[col] = df_clean[col].str.strip()

# Drop duplicates
df_clean = df_clean.drop_duplicates()

# Fill missing categorical with "Unknown"
for col in ["collision_type", "property_damage", "police_report_available"]:
    if col in df_clean.columns:
        df_clean[col] = df_clean[col].fillna("Unknown")

print("Cleaned shape:", df_clean.shape)
df_clean.head()


Cleaned shape: (1000, 39)


,months_as_customer,age,policy_number,policy_bind_date,policy_state,policy_csl,policy_deductable,policy_annual_premium,umbrella_limit,insured_zip,insured_sex,insured_education_level,insured_occupation,insured_hobbies,insured_relationship,capital-gains,capital-loss,incident_date,incident_type,collision_type,incident_severity,authorities_contacted,incident_state,incident_city,incident_location,incident_hour_of_the_day,number_of_vehicles_involved,property_damage,bodily_injuries,witnesses,police_report_available,total_claim_amount,injury_claim,property_claim,vehicle_claim,auto_make,auto_model,auto_year,fraud_reported
0,328,48,521585,2014-10-17,OH,250/500,1000,1406.91,0,466132,MALE,MD,craft-repair,sleeping,husband,53300,0,2015-01-25,Single Vehicle Collision,Side Collision,Major Damage,Police,SC,Columbus,9935 4th Drive,5,1,YES,1,2,YES,71610,6510,13020,52080,Saab,92x,2004,Y
1,228,42,342868,2006-06-27,IN,250/500,2000,1197.22,5000000,468176,MALE,MD,machine-op-inspct,reading,other-relative,0,0,2015-01-21,Vehicle Theft,Unknown,Minor Damage,Police,VA,Riverwood,6608 MLK Hwy,8,1,Unknown,0,0,Unknown,5070,780,780,3510,Mercedes,E400,2007,Y
2,134,29,687698,2000-09-06,OH,100/300,2000,1413.14,5000000,430632,FEMALE,PhD,sales,board-games,own-child,35100,0,2015-02-22,Multi-vehicle Collision,Rear Collision,Minor Damage,Police,NY,Columbus,7121 Francis Lane,7,3,NO,2,3,NO,34650,7700,3850,23100,Dodge,RAM,2007,N
3,256,41,227811,1990-05-25,IL,250/500,2000,1415.74,6000000,608117,FEMALE,PhD,armed-forces,board-games,unmarried,48900,-62400,2015-01-10,Single Vehicle Collision,Front Collision,Major Damage,Police,OH,Arlington,6956 Maple Drive,5,1,Unknown,1,2,NO,63400,6340,6340,50720,Chevrolet,Tahoe,2014,Y
4,228,44,367455,2014-06-06,IL,500/1000,1000,1583.91,6000000,610706,MALE,Associate,sales,board-games,unmarried,66000,-46000,2015-02-17,Vehicle Theft,Unknown,Minor Damage,NaN,NY,Arlington,3041 3rd Ave,20,1,NO,0,1,NO,6500,1300,650,4550,Accura,RSX,2009,N


In [23]:
#Feature Engineering
#Time-Based Features

df_feat = df_clean.copy()

df_feat["incident_day_of_week"] = df_feat["incident_date"].dt.day_name()
df_feat["incident_month"] = df_feat["incident_date"].dt.month
df_feat["incident_hour_bin"] = pd.cut(
    df_feat["incident_hour_of_the_day"],
    bins=[-1, 5, 11, 17, 23],
    labels=["Late Night", "Morning", "Afternoon", "Evening"]
)
df_feat["customer_tenure_years"] = (df_feat["months_as_customer"] / 12).round(1)

# Recency feature
df_feat = df_feat.sort_values(["policy_number", "incident_date"])
df_feat["days_since_last_claim"] = df_feat.groupby("policy_number")["incident_date"].diff().dt.days
df_feat["days_since_last_claim"] = df_feat["days_since_last_claim"].fillna(-1)

#Statistical Features
claims = ["injury_claim", "property_claim", "vehicle_claim", "total_claim_amount"]

df_feat["claim_mean"] = df_feat[claims].mean(axis=1)
df_feat["claim_std_dev"] = df_feat[claims].std(axis=1)
df_feat["claim_percentile_rank"] = df_feat["total_claim_amount"].rank(pct=True)

# IQR flag
Q1 = df_feat["total_claim_amount"].quantile(0.25)
Q3 = df_feat["total_claim_amount"].quantile(0.75)
IQR = Q3 - Q1
df_feat["iqr_flag"] = ((df_feat["total_claim_amount"] < (Q1 - 1.5*IQR)) |
                       (df_feat["total_claim_amount"] > (Q3 + 1.5*IQR))).astype(int)


In [25]:
#Anomaly Detection

# Z-score
mean_claim = df_feat["total_claim_amount"].mean()
std_claim = df_feat["total_claim_amount"].std()
df_feat["zscore_total_claim"] = (df_feat["total_claim_amount"] - mean_claim) / std_claim
df_feat["anomaly_zscore"] = (df_feat["zscore_total_claim"].abs() > 3).astype(int)

# Benford’s Law
def leading_digit(x):
    s = str(int(abs(x))) if not pd.isna(x) else ""
    return int(s[0]) if s else np.nan

df_feat["leading_digit"] = df_feat["total_claim_amount"].apply(leading_digit)
benford = {1: 0.301, 2: 0.176, 3: 0.125, 4: 0.097, 5: 0.079, 6: 0.067, 7: 0.058, 8: 0.051, 9: 0.046}
df_feat["benford_expected"] = df_feat["leading_digit"].map(benford)
df_feat["benford_flag"] = (df_feat["benford_expected"] < 0.05).fillna(False).astype(int)


In [29]:
#Rule Flags

#Unusual hour (1–5 AM)
df_feat["rule_unusual_hour"] = df_feat["incident_hour_of_the_day"].between(1, 5).astype(int)

# High frequency (>3 claims within a 30-day window per policy)
def add_rolling_counts(g: pd.DataFrame) -> pd.DataFrame:
    g = g.sort_values("incident_date").set_index("incident_date")
    # count rows over a rolling 30-day window
    rolling_counts = pd.Series(1, index=g.index).rolling("30D").sum()
    g["claims_last_30d"] = rolling_counts.values
    return g.reset_index()

#Use include_groups=False to silence the deprecation warning (pandas ≥2.2)
try:
    df_feat = (
        df_feat
        .groupby("policy_number", group_keys=False)
        .apply(add_rolling_counts, include_groups=False)
    )
except TypeError:
#Fallback for older pandas that don't support include_groups
    df_feat = (
        df_feat
        .groupby("policy_number", group_keys=False)
        .apply(add_rolling_counts)
    )

df_feat["rule_high_frequency"] = (df_feat["claims_last_30d"] > 3).astype(int)

#Suspicious missing info
df_feat["rule_suspicious_missing"] = (
    (df_feat["police_report_available"] == "Unknown") |
    (df_feat["property_damage"] == "Unknown")
).astype(int)


In [31]:
#Document Data Dictionary
data_dict = pd.DataFrame([
    ("incident_day_of_week","Day of week of incident","engineered"),
    ("incident_month","Month of incident","engineered"),
    ("incident_hour_bin","Time of day category","engineered"),
    ("customer_tenure_years","Customer tenure in years","engineered"),
    ("days_since_last_claim","Days since prior claim (-1 = first claim)","engineered"),
    ("claim_mean","Mean of injury/property/vehicle claims","engineered"),
    ("claim_std_dev","Standard deviation of claims","engineered"),
    ("claim_percentile_rank","Percentile rank of total claim","engineered"),
    ("iqr_flag","IQR outlier flag","anomaly_flag"),
    ("zscore_total_claim","Z-score of total claim","anomaly"),
    ("anomaly_zscore","Flag: |Z|>3","anomaly_flag"),
    ("leading_digit","Leading digit (Benford)","anomaly"),
    ("benford_expected","Expected Benford probability","anomaly"),
    ("benford_flag","Benford anomaly flag","anomaly_flag"),
    ("rule_unusual_hour","Flag: claim at 1-5 AM","rule_flag"),
    ("claims_last_30d","Rolling 30-day claims count","rule_feature"),
    ("rule_high_frequency","Flag: >3 claims in 30 days","rule_flag"),
    ("rule_suspicious_missing","Flag: missing police/property info","rule_flag"),
], columns=["Field","Description","Type"])

data_dict


,Field,Description,Type
0,incident_day_of_week,Day of week of incident,engineered
1,incident_month,Month of incident,engineered
2,incident_hour_bin,Time of day category,engineered
3,customer_tenure_years,Customer tenure in years,engineered
4,days_since_last_claim,Days since prior claim (-1 = first claim),engineered
5,claim_mean,Mean of injury/property/vehicle claims,engineered
6,claim_std_dev,Standard deviation of claims,engineered
7,claim_percentile_rank,Percentile rank of total claim,engineered
8,iqr_flag,IQR outlier flag,anomaly_flag
9,zscore_total_claim,Z-score of total claim,anomaly


In [33]:
#Save Outputs
df_clean.to_csv("insurance_claims_cleaned.csv", index=False)
df_feat.to_csv("insurance_claims_featured.csv", index=False)
data_dict.to_csv("data_dictionary_engineered.csv", index=False)

print("Saved: insurance_claims_cleaned.csv, insurance_claims_featured.csv, data_dictionary_engineered.csv")


Saved: insurance_claims_cleaned.csv, insurance_claims_featured.csv, data_dictionary_engineered.csv
